In [ ]:
import ipywidgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg, Heartbeat
from IPython.display import display
import threading
import time

# Controller 
controller = widgets.Controller(index=0)
display(controller)

# Robot 
robot = Robot()

# Camera 
cam_width = 320
cam_height = 240
camera = Camera.instance(width=cam_width, height=cam_height)
image = widgets.Image(format='jpeg', width=cam_width, height=cam_height)
display(image)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# Heartbeat stop 
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        #camera_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
heartbeat.observe(handle_heartbeat_status, names='status')

# Arcade drive in background thread 
def arcade_loop():
    while True:
        if len(controller.axes) >= 2:
            forward_scale = 0.5  # axis 1 mapped to [-0.5, 0.5]
            turn_scale    = 0.2  # axis 0 mapped to [-0.2, 0.2]
            forward = -controller.axes[1].value * forward_scale
            turn    = controller.axes[0].value * turn_scale
            left_motor  = max(-1, min(1, forward + turn))
            right_motor = max(-1, min(1, forward - turn))
            robot.left_motor.value  = left_motor
            robot.right_motor.value = right_motor
        time.sleep(0.02)  # 50 Hz update

thread = threading.Thread(target=arcade_loop, daemon=True)
thread.start()
